# *Mirror*

### Il Concetto

Invece di calcolare la luce (come le Probe), usiamo una "Telecamera Spia".

1. Una **Seconda Camera** guarda ciò che dovrebbe riflettere lo specchio.
2. Invia l'immagine a una **Texture** (invece che allo schermo).
3. La Texture viene applicata sulla **Superficie dello specchio**.
### Procedura Passo-Passo

#### 1. Prepara la "Pellicola" (Render Texture)

Questa è la texture che riceverà il video in diretta.

1. Nel pannello **Project**, fai tasto destro > **Create** > **Render Texture**.
2. Chiamala `MirrorTexture`.
3. **Importante:** Nell'Inspector, cambia la **Size** (risoluzione). Inizia con `512 x 512` o `1024 x 1024`. Più è alta, più lo specchio è nitido, ma più è pesante per il computer.

#### 2. Prepara lo "Specchio" (L'Oggetto)

1. Crea un oggetto 3D nella scena (es. `3D Object > Quad` o `Plane`). Chiamalo `MirrorObject`.
2. Posizionalo dove vuoi (es. appeso al muro).
3. Crea un nuovo **Materiale** (`Create > Material`), chiamalo `MirrorMat`.
4. Trascina la tua `MirrorTexture` (creata al punto 1) sopra lo slot **Albedo** (o Base Map) del materiale.
5. Trascina questo materiale sopra il tuo `MirrorObject`.
* *Nota:* Per ora lo vedrai probabilmente nero o grigio. È normale.

#### 3. Prepara l' "Occhio" (La Reflection Camera)

1. Crea una nuova Camera: `GameObject > Camera`. Chiamala `MirrorCam`.
2. Posiziona la `MirrorCam` esattamente al centro del tuo specchio.
3. **Il trucco:** Ruota la telecamera di 180 gradi sull'asse Y (o in modo che guardi *fuori* dallo specchio, verso il giocatore).
4. Nell'Inspector della Camera, trova la voce **Target Texture**.
5. Trascina la tua `MirrorTexture` dentro questo slot.
* *Risultato:* Ora la videocamera non mostrerà più nulla sullo schermo del gioco (Game View), ma starà "trasmettendo" sulla superficie dello specchio.

#### 4. Risolvere il problema dell'inversione (Il "Caveat")

Come citava il testo, a questo punto avrai un'immagine, ma sarà come una videochiamata (se alzi la mano destra, lo specchio mostra la mano alla sua sinistra). Uno specchio deve essere invertito.

Hai due modi per farlo rapidamente senza codice complesso:

* **Metodo A (Semplice):** Seleziona il tuo `MirrorObject` (il Quad/Plane) e imposta la **Scale X** a **-1**. Questo capovolge fisicamente la superficie, invertendo l'immagine.
* **Metodo B (Camera):** Nelle impostazioni della `MirrorCam`, sotto "Projection", a volte si può giocare con le matrici, ma il metodo A è il più rapido per i principianti.

### I Limiti di questo sistema (Senza Script)

Il metodo sopra crea quello che sembra uno specchio o una telecamera di sicurezza (CCTV). Tuttavia, ha un difetto fisico: **La prospettiva è fissa.**

Se il giocatore si sposta lateralmente, l'immagine riflessa non cambia angolazione correttamente (perché la telecamera dello specchio è ferma).

Per avere uno specchio perfetto che cambia prospettiva mentre ti muovi (come in *GTA* o giochi moderni), serve un piccolo **Script C#**.

#### Script Base per Specchio (Opzionale ma consigliato)

Se vuoi provare, crea uno script chiamato `MirrorCamera.cs` e attaccalo alla `MirrorCam`. Questo script dice alla telecamera dello specchio di comportarsi in modo relativo alla telecamera del giocatore.

```csharp
using UnityEngine;

public class MirrorCamera : MonoBehaviour
{
    public Transform playerCamera; // Trascina qui la Main Camera
    public Transform mirrorPlane;  // Trascina qui il MirrorObject

    void Update()
    {
        // Calcola la posizione riflessa rispetto allo specchio
        Vector3 localPlayerPos = mirrorPlane.InverseTransformPoint(playerCamera.position);
        
        // Inverte la posizione sull'asse che esce dallo specchio (solitamente Z o Y a seconda del Quad/Plane)
        // Per un Quad verticale standard è spesso l'asse Z locale
        Vector3 lookAtPos = mirrorPlane.TransformPoint(new Vector3(localPlayerPos.x, localPlayerPos.y, -localPlayerPos.z));

        // La camera dello specchio guarda verso il punto riflesso
        transform.LookAt(lookAtPos);
    }
}

```

*(Nota: Gli script per specchi perfetti possono diventare complessi per gestire la "Oblique Frustum", ovvero il taglio dell'immagine ai bordi dello specchio, ma questo è il concetto base).*

### Riepilogo Performance

Ricorda l'avvertimento del testo: **Usare una seconda camera significa che Unity deve disegnare il mondo due volte.**
Se il tuo gioco va a 60 FPS, aggiungere uno specchio potrebbe farlo scendere a 30-40 FPS se la scena è complessa.

* **Consiglio:** Tieni la risoluzione della Render Texture bassa (256 o 512) se lo specchio è piccolo o poco importante.